In [ ]:
import numpy as np
import pandas as pd
import fastf1
from src.utils import compute_track_dominance_multi, quali_track_evolution
from src.plotset import setup_plot, save_fig, plot_track_dominance, plot_quali_track_evolution

from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,15,'Q')
session.load()

In [ ]:
points, segments, colors = compute_track_dominance_multi(session=session,drivers=['PIA','NOR'],circuit_length=4259,
                                                         colors_map={'PIA':'#FF8000','NOR':'#FFFFFF'},window_size=100)

In [ ]:
fig = plot_track_dominance(points=points, segments=segments, colors=colors,figsize=(12,11))

In [ ]:
save_fig(fig=fig, name='track_dominance', loc='Reel14')

In [ ]:
df = quali_track_evolution(session=session)

In [ ]:
round(df.Q2.mean() - df.Q1.mean(),1), round(df.Q3.mean() - df.Q2.mean(),1)

In [ ]:
setup_plot(xyticksize=22,axeslabel=24,figtitle=30)
fig = plot_quali_track_evolution(df)

In [ ]:
save_fig(fig=fig, name='quali_evolution', loc='Reel14')

In [ ]:
team_delta = pd.read_csv('q_delta.csv')

In [ ]:
team_delta['colors'] = [plotting.get_team_color(session=session,identifier=name) for name in team_delta.Team]

In [ ]:
color_dict = {key: item for key, item in team_delta[['Team','colors']].values}

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.barplot(data=team_delta,x='Team',y='delta',hue='Team',palette=color_dict)

ax.set_ylim(-3,0.5)

ax.set_xlabel('')
ax.set_xticks([])
ax.set_ylabel('Lap Time Delta (s)')
ax.set_title('Lap Time Improvement vs 2024 (-ve is better)')

In [ ]:
save_fig(fig=fig, name='team_delta', loc='Reel14')

In [ ]:
# Compute 2024 gap relative to fastest team (McLaren)
team_delta['2024_gap'] = team_delta['2024'] - team_delta['2024'].min()

# Sort by 2025 lap times (ascending = fastest to slowest)
team_delta_sorted = team_delta.sort_values('2025')

fig, ax = plt.subplots(figsize=(15,8))

# Plot bars with baseline = 2024 gap, height = delta, team colors
for i, row in team_delta_sorted.iterrows():
    ax.bar(
        row['Team'],
        row['delta'],
        bottom=row['2024_gap'],
        color=color_dict.get(row['Team'], 'gray')
    )
    # Add delta text label below each bar
    ax.text(
        row['Team'],
        row['2024_gap'] + row['delta'] - 0.1,   # slightly below bar end
        f"{row['delta']:.2f}s",
        ha='center',
        va='top',
        color='white',
        fontsize=20,
        fontweight='bold'
    )

ax.set_ylim(-2,3)

# ax.set_ylabel('Lap Time (s)')
ax.set_xlabel('')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Lap Time Improvement w.r.t 2024')

plt.show()

In [ ]:
save_fig(fig=fig, name='team_delta', loc='Reel14')